### Estadisticas
#### 1 sample 1 epoch gpu 1
tarda de media 30 segundos.
#### 2 sample 1 epoch gpu 1
tarda de media 58 segundos.
#### 2 sample 1 epoch gpu 0.5
tarda de media 30 segundos. (y la gpu esta al 44% y le sobra memoria.)
#### 5 sample 1 epoch gpu 0.2
ocurren errores por memoria insuficiente en la ram (15.4 disponible para uso, del programa, y no alcanza.)
#### 4 sample 1 epoch gpu 0.25
tarda de media 1 minuto con 13 segundos, utiliza 11 gb de memoria ram, la gpu esta a mitad de capacidad de computo.
la cantidad de memoria ram es un cuello de botella para utilizar todo el poder de la GPU.
#### 4 sample 10 epoch gpu 0.25
tarda de media 2 minutos con 11 segundos, pero hay que tener cuidado ya que mientras entrena consume poca memoria, pero al momento de validar su loss y acc incrementa al nivel de warning con la RAM.
#### 3 sample 10 epoch gpu 0.33333
tarda 1 minuto y 40 segundos, pero con redes grandes tambien llega al warning de RAM.
#### 2 sample 10 epoch gpu 0.5
58 segundos, no se acerca a warning de RAM.
#### 10 sample 10 epoch gpu 0.5
4 minutos 25 segundos, corriendo de a dos modelos a la vez, maxima cantidad de memoria RAM usada 12.5 GB

2 configuraciones llegaron hasta las 10 epoch, 1 hasta la octava, el resto tuvo un desempeño pobre  desde el inicio y ray tune (programador ASHA) los termino en la primera epoch (se puede configurar la cantidad minima de epoch de ejecucion antes de terminarlas).

### Probar buscar hiperparametros de la red.
se deja estaticos el learning rate y el batch size. a 1-e4 y 16, respectivamente.

El resto se prueba buscar, para ver el tiempo y rendimiento se prueba 10 samples 50 epoch gpu 0.5

el resultado es una media de 14 minutos.


In [4]:
# python
import os
from functools import partial
# pytorch
import torch
from torch import nn
# pytorch utilidades
from torch.utils.data import random_split
# ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
# yo
from cargar_datos import cargar_datasets, cargar_dataloaders
from modelo_convolucional import instanciarRed


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0):
    data_dir = os.path.abspath("./data")
    checkpoint_dir = os.path.abspath("./checkpoint_dir")

    # se carga una vez, para que si no esta descargado, se descarge y se valide, el resto de veces en las pruebas, se saltara esta parte.
    cargar_datasets(data_dir)
    """
    variable de configuracion, Ray tune modificara sus valores en el proceso de ajuste de hiperparametros.
    """
    config = {
        #"batch_size": tune.grid_search([8, 16, 32]),
        # "learning_rate": tune.loguniform(1e-4, 1e-1),
        "batch_size": 16,
        "learning_rate": 1e-4,
        # red convolucional
        "cant_filtros_conv1": tune.grid_search([6, 12, 18]),
        # "kernel_size_maxpool1": tune.grid_search([2, 3]),
        # "cant_filtros_conv2": tune.grid_search([16, 22, 28]),
        # "kernel_size_maxpool2": tune.grid_search([2, 3]),
        # "full_l1": tune.grid_search([120, 140, 160]),
        # "full_l2": tune.grid_search([84, 104, 124])
        "kernel_size_maxpool1": 2,
        "cant_filtros_conv2": 16,
        "kernel_size_maxpool2": 2,
        "full_l1": 120,
        "full_l2": 84
    }
    """
    configurar Ray y correr las pruebas en busqueda de hiperparametros
    """
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=list(config.keys()),
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        # partial es una funcion de orden superior de tipo curry
        partial(train_cifar, data_dir=data_dir, checkpoint_dir=checkpoint_dir, max_num_epochs=max_num_epochs),
        resources_per_trial={"cpu": 4, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    """
    Validar el desempeño de la red con mejor puntuacion en la busqueda de hiperparametros contra el conjunto de datos de validacion.
    """
    best_trained_model = instanciarRed(best_trial.config)
    device = getDevice()
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.dir_or_data
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device, data_dir)
    print("Best trial test set accuracy: {}".format(test_acc))


def test_accuracy(model, device="cpu", data_dir="./data"):
    _, testset = cargar_datasets(data_dir)
    testloader = cargar_dataloaders(testset, 4)

    correct = 0
    total = 0
    # pone a la red en moodo evaluacion, desactiva capas dropout.
    model.eval()
    # desactiva el proceso de calculo y guardado de valores intermedios
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


def getDevice():
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            raise Exception(
                "la pc cuenta con multiples gpus, deberia utilizar DataParallel")
            #model = nn.DataParallel(model)
    return device


def train_cifar(config, checkpoint_dir=None, data_dir=None,max_num_epochs=10) :
    model = instanciarRed(config)

    device = getDevice()
    # le dice a Pytorch donde debe ejecutar el modelo.
    model.to(device)
    # CrossEntropyLoss requiere el output "logits", no es necesario pasarlo por el softmax, ya que lo calcula dentro. https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    criterion = nn.CrossEntropyLoss()

    # se necesita enviar los parametros del modelo al optimizador para que los pueda actualizar.
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["learning_rate"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, _ = cargar_datasets(data_dir)
    # reservar una parte de los datos de entrenamiento para validacion.
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = cargar_dataloaders(train_subset, config["batch_size"])
    valloader = cargar_dataloaders(val_subset, config["batch_size"])

    for epoch in range(max_num_epochs):  # loop over the dataset multiple times

        train_loop(model, device, trainloader, criterion, optimizer, epoch)
        loss, accuracy = validation_loss(model, device, valloader, criterion)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)
    print("Finished Training")


def train_loop(model, device, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_steps += 1
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0


def validation_loss(net, device, valloader, criterion):
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    net.eval()
    for i, data in enumerate(valloader, 0):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.cpu().numpy()
            val_steps += 1
    loss = val_loss / val_steps
    accuracy = correct / total
    return loss, accuracy


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    """
    si config posee tune.grid_search, en alguno parametro, entonces num_samples contendra no la cantidad de ejemplos que le decis en num_samples, sino num_samples *(combinatoria grid search)
    ej: dos parametros grid search con 3 y 4 elementos, sus combinaciones son 3*4 = 12, si num_samples es 3 entonces es 12*3 = 36.
    """
    main(num_samples=1, max_num_epochs=3, gpus_per_trial=0.5)


Files already downloaded and verified
Files already downloaded and verified
== Status ==
Current time: 2022-11-19 16:59:25 (running for 00:00:00.15)
Memory usage on this node: 4.9/15.4 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 2.000: None | Iter 1.000: None
Resources requested: 4.0/20 CPUs, 0.5/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+
| Trial name              | status   | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |
|-------------------------+----------+---------------------+---------

0.0%(func pid=39322) 
0.0%(func pid=39356) 
0.0%(func pid=39322) 
0.0%(func pid=39356) 
0.1%(func pid=39322) 
0.1%(func pid=39356) 
0.2%(func pid=39322) 
0.2%(func pid=39356) 
0.4%(func pid=39322) 
0.5%(func pid=39356) 
0.8%(func pid=39322) 
1.0%(func pid=39356) 
1.1%(func pid=39356) 
1.2%(func pid=39356) 
0.9%(func pid=39322) 
2.0%(func pid=39356) 
1.6%(func pid=39322) 
2.1%(func pid=39356) 
1.6%(func pid=39322) 
2.1%(func pid=39356) 
3.3%(func pid=39356) 
1.7%(func pid=39322) 
3.4%(func pid=39356) 
3.5%(func pid=39356) 
3.6%(func pid=39356) 
2.8%(func pid=39322) 
3.9%(func pid=39356) 
2.9%(func pid=39322) 
5.1%(func pid=39356) 
5.2%(func pid=39356) 
5.3%(func pid=39356) 
5.4%(func pid=39356) 
5.4%(func pid=39356) 
5.4%(func pid=39356) 
5.5%(func pid=39356) 
4.2%(func pid=39322) 
5.9%(func pid=39356) 
4.3%(func pid=39322) 
6.8%(func pid=39356) 
4.4%(func pid=39322) 
7.1%(func pid=39356) 
5.8%(func pid=39322) 
8.9%(func pid=39356) 
6.1%(func pid=39322) 
6.2%(func pid=39322) 
10.1%(func

== Status ==
Current time: 2022-11-19 16:59:36 (running for 00:00:11.17)
Memory usage on this node: 7.1/15.4 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 2.000: None | Iter 1.000: None
Resources requested: 8.0/20 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (1 PENDING, 2 RUNNING)
+-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+
| Trial name              | status   | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |
|-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----

10.5%(func pid=39322) 
15.7%(func pid=39356) 
10.6%(func pid=39322) 
11.4%(func pid=39322) 
16.4%(func pid=39356) 
16.4%(func pid=39356) 
16.4%(func pid=39356) 
11.8%(func pid=39322) 
17.2%(func pid=39356) 
11.9%(func pid=39322) 
12.5%(func pid=39322) 
18.0%(func pid=39356) 
13.0%(func pid=39322) 
18.6%(func pid=39356) 
13.0%(func pid=39322) 
13.1%(func pid=39322) 
18.7%(func pid=39356) 
13.1%(func pid=39322) 
13.2%(func pid=39322) 
18.7%(func pid=39356) 
13.2%(func pid=39322) 
18.7%(func pid=39356) 
13.2%(func pid=39322) 
13.6%(func pid=39322) 
19.7%(func pid=39356) 
19.8%(func pid=39356) 
19.9%(func pid=39356) 
14.3%(func pid=39322) 
20.3%(func pid=39356) 
14.4%(func pid=39322) 
20.3%(func pid=39356) 
14.8%(func pid=39322) 
20.8%(func pid=39356) 
20.9%(func pid=39356) 
15.5%(func pid=39322) 
21.9%(func pid=39356) 
15.6%(func pid=39322) 
21.9%(func pid=39356) 
15.6%(func pid=39322) 
15.7%(func pid=39322) 
16.0%(func pid=39322) 
22.6%(func pid=39356) 
22.6%(func pid=39356) 
22.7%(func 

== Status ==
Current time: 2022-11-19 16:59:41 (running for 00:00:16.18)
Memory usage on this node: 7.1/15.4 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 2.000: None | Iter 1.000: None
Resources requested: 8.0/20 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (1 PENDING, 2 RUNNING)
+-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+
| Trial name              | status   | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |
|-------------------------+----------+---------------------+--------------+-----------------+----------------------+------------------------+----

...

== Status ==
Current time: 2022-11-19 17:00:23 (running for 00:00:58.29)
Memory usage on this node: 5.8/15.4 GiB 
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 2.000: -1.6772695169448852 | Iter 1.000: -1.8262733119010925
Resources requested: 4.0/20 CPUs, 0.5/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (1 RUNNING, 2 TERMINATED)
+-------------------------+------------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |    loss |   accuracy |   training_iteration |
|---------------

92.8%(func pid=41408) 
93.7%(func pid=41408) 
94.6%(func pid=41408) 
95.5%(func pid=41408) 
95.6%(func pid=41408) 
95.7%(func pid=41408) 
95.7%(func pid=41408) 
95.9%(func pid=41408) 
96.0%(func pid=41408) 
96.1%(func pid=41408) 
96.2%(func pid=41408) 
96.5%(func pid=41408) 
97.3%(func pid=41408) 
97.4%(func pid=41408) 
97.4%(func pid=41408) 
97.5%(func pid=41408) 
98.6%(func pid=41408) 
99.7%(func pid=41408) 
100.0%36m(func pid=41408) 


(func pid=41408) Extracting ./data/cifar-10-python.tar.gz to ./data
(func pid=41408) Files already downloaded and verified
== Status ==
Current time: 2022-11-19 17:00:28 (running for 00:01:03.29)
Memory usage on this node: 7.0/15.4 GiB 
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 2.000: -1.6772695169448852 | Iter 1.000: -1.8262733119010925
Resources requested: 4.0/20 CPUs, 0.5/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (1 RUNNING, 2 TERMINATED)
+-------------------------+------------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filt

2022-11-19 17:00:43,619	INFO tune.py:777 -- Total run time: 78.35 seconds (78.24 seconds for the tuning loop).


== Status ==
Current time: 2022-11-19 17:00:43 (running for 00:01:18.24)
Memory usage on this node: 8.1/15.4 GiB 
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 2.000: -1.6414100234031679 | Iter 1.000: -1.7864445062637329
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/braian/ray_results/train_cifar_2022-11-19_16-59-25
Number of trials: 3/3 (3 TERMINATED)
+-------------------------+------------+---------------------+--------------+-----------------+----------------------+------------------------+----------------------+------------------------+-----------+-----------+---------+------------+----------------------+
| Trial name              | status     | loc                 |   batch_size |   learning_rate |   cant_filtros_conv1 |   kernel_size_maxpool1 |   cant_filtros_conv2 |   kernel_size_maxpool2 |   full_l1 |   full_l2 |    loss |   accuracy |   training_iteration |
|-------------------------+----